In [1]:
import matplotlib.pyplot as plt
import numpy as np
from random import uniform, randint, shuffle
from NPSmethods2 import readInImages, pupilFunc, getFreq
from numpy.fft import ifftshift, irfft2
import pickle as pkl
import matplotlib.cm as cm
from komm import AWGNChannel
from sklearn.preprocessing import QuantileTransformer, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#Geranate random assortment of parameters
def make_Paras(seedParasMin, seedParasMax, numToGen):
    outPara = np.empty(shape =(numToGen, len(seedParasMin)))
    paraTemp = [None] * len(seedParasMin)
    for i in range(numToGen):
        for j in range(len(seedParasMin)):
            if j == 5:
                if seedParasMin[j] < 0:
                    low = 1.2
                else:
                    low = 0.83
                if seedParasMax[j] < 0:
                    high = 0.83
                else:
                    high = 1.2
            else:
                low = 1
                high = 1
            para = uniform(seedParasMin[j] * low, seedParasMax[j] * high)
            paraTemp[j] = para
        outPara[i] = paraTemp
    return outPara
        

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = getFreq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

#Generate NPS noise
# def generate_Noise(M2k_Fit):
#     M2k_Fit = np.delete(M2k_Fit, 0, axis=0)
#     M2k_Fit = np.delete(M2k_Fit, 0, axis=1)

#     NPSavgs = np.zeros(shape=np.shape(M2k_Fit))
#     NSR = np.random.normal(loc=0.1, 
#                            scale=0.2, 
#                            size=(99,50))
#     NSR = Mirror(NSR)
    
#     bkg_noise = np.random.normal(loc=0.0259, 
#                                  scale=0.00616, 
#                                  size=(99,50))
#     bkg_noise = Mirror(bkg_noise)
#     M2k_Fit = M2k_Fit + bkg_noise

#     NPSavgs = M2k_Fit + M2k_Fit * NSR
#     NPSavgs[NPSavgs < 0] = 0
    
#     return NPSavgs

def Get_Max(array):
    max1 = np.amax(array) * 3
    max2 = np.amax(array[array < max1]) * 3
    array[np.where(array >= max2)] = np.mean(array)
    
    return array, max1, max2
    
def Mirror(array):
    array2 = np.flip(np.flip(array, 0), 1)
    array2 = np.delete(array2, (0), axis=1)
    newarray = np.concatenate((array, array2), 1)
    
    return newarray

#Turn one NPS to one OD
def NPSavg_To_Single(NPSavgs, numNPSs):
    NPSs = []
    NPSnum = 0
    NPSavgs = np.delete(NPSavgs, np.s_[75:100], axis = 1)
    NPSavgs = np.delete(NPSavgs, np.s_[0:26], axis = 1)
    NPSavgs = np.delete(NPSavgs, np.s_[75:100], axis = 2)
    NPSavgs = np.delete(NPSavgs, np.s_[0:26], axis = 2)
        
    for NPSavg in NPSavgs:
        NPSnum += 1
        print("Parameters Set: %d" % NPSnum)
        for i in range(numNPSs):
            NPStemp = NPSavg.copy()
            NSR = np.random.exponential(scale=1.1, size=(49,25)) - 1
            NSR, max1, max2 = Get_Max(NSR)
            NSR = Mirror(NSR)
            Bkg = np.random.exponential(scale=0.032, size=(49,25))
            Bkg = Mirror(Bkg)
            
            randc = randint(1,3) // 3
            randx = randint(1,3) // 2
            randy = randint(1,3) // 2
            
            mean = np.mean(NPStemp)
            amax = np.amax(NPStemp)
                        
            for j in range(len(NPStemp)):
                for k in range(len(NPStemp.T)):
                    if j != 24 and i != 24:
                        NPStemp[j][k] += Bkg[j][k]
                    if ((j == 23 or j == 25) and k == 24):
                        if randc == 1:
                            NPStemp[j][k] = mean + mean * (max2)
                        else :
                            NPStemp[j][k] = amax + amax * (max1)
                    elif (j == 24 and (k == 23 or k == 25)):
                        if randc == 1:
                            NPStemp[j][k] = amax + amax * (max1)
                        else:
                            NPStemp[j][k] = mean + mean * (max2)
                    else:
                        NPStemp[j][k] += NPStemp[j][k] * NSR[j][k]
                    if j == 24:
                        NPStemp[j][k] += mean * randx
                    if k == 24:
                        NPStemp[j][k] += mean * randy
            NPSs.append(NPStemp)
            
    return NPSs

In [3]:
savepath = './results/'
# seedParas = [0.35060326843236184, 
#              0.6484441665068852, 
#              -2.63564159087613,
#              0.5094606166480791, 
#              -1.8773441762223027, 
#              0.8206242586655179, 
#              1.0257364487180909]

numNPSs = 5
numParas = 20000

seedParasMax = pkl.load(open(savepath + "seedParasMax.pkl", "rb"))
seedParasMin = pkl.load(open(savepath + "seedParasMin.pkl", "rb"))

NPSavgs = []
fakeNPSs = []
savedParas = []

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
} 

for paras in make_Paras(seedParasMin, seedParasMax, numParas):
    NPSavgs.append(make_M2k_Fit(paras, imgSysData))
    for j in range(numNPSs):
        savedParas.append(paras)
fakeNPSs = NPSavg_To_Single(NPSavgs, numNPSs)

Parameters Set: 1
Parameters Set: 2
Parameters Set: 3
Parameters Set: 4
Parameters Set: 5
Parameters Set: 6
Parameters Set: 7
Parameters Set: 8
Parameters Set: 9
Parameters Set: 10
Parameters Set: 11
Parameters Set: 12
Parameters Set: 13
Parameters Set: 14
Parameters Set: 15
Parameters Set: 16
Parameters Set: 17
Parameters Set: 18
Parameters Set: 19
Parameters Set: 20
Parameters Set: 21
Parameters Set: 22
Parameters Set: 23
Parameters Set: 24
Parameters Set: 25
Parameters Set: 26
Parameters Set: 27
Parameters Set: 28
Parameters Set: 29
Parameters Set: 30
Parameters Set: 31
Parameters Set: 32
Parameters Set: 33
Parameters Set: 34
Parameters Set: 35
Parameters Set: 36
Parameters Set: 37
Parameters Set: 38
Parameters Set: 39
Parameters Set: 40
Parameters Set: 41
Parameters Set: 42
Parameters Set: 43
Parameters Set: 44
Parameters Set: 45
Parameters Set: 46
Parameters Set: 47
Parameters Set: 48
Parameters Set: 49
Parameters Set: 50
Parameters Set: 51
Parameters Set: 52
Parameters Set: 53
Pa

Parameters Set: 418
Parameters Set: 419
Parameters Set: 420
Parameters Set: 421
Parameters Set: 422
Parameters Set: 423
Parameters Set: 424
Parameters Set: 425
Parameters Set: 426
Parameters Set: 427
Parameters Set: 428
Parameters Set: 429
Parameters Set: 430
Parameters Set: 431
Parameters Set: 432
Parameters Set: 433
Parameters Set: 434
Parameters Set: 435
Parameters Set: 436
Parameters Set: 437
Parameters Set: 438
Parameters Set: 439
Parameters Set: 440
Parameters Set: 441
Parameters Set: 442
Parameters Set: 443
Parameters Set: 444
Parameters Set: 445
Parameters Set: 446
Parameters Set: 447
Parameters Set: 448
Parameters Set: 449
Parameters Set: 450
Parameters Set: 451
Parameters Set: 452
Parameters Set: 453
Parameters Set: 454
Parameters Set: 455
Parameters Set: 456
Parameters Set: 457
Parameters Set: 458
Parameters Set: 459
Parameters Set: 460
Parameters Set: 461
Parameters Set: 462
Parameters Set: 463
Parameters Set: 464
Parameters Set: 465
Parameters Set: 466
Parameters Set: 467


Parameters Set: 834
Parameters Set: 835
Parameters Set: 836
Parameters Set: 837
Parameters Set: 838
Parameters Set: 839
Parameters Set: 840
Parameters Set: 841
Parameters Set: 842
Parameters Set: 843
Parameters Set: 844
Parameters Set: 845
Parameters Set: 846
Parameters Set: 847
Parameters Set: 848
Parameters Set: 849
Parameters Set: 850
Parameters Set: 851
Parameters Set: 852
Parameters Set: 853
Parameters Set: 854
Parameters Set: 855
Parameters Set: 856
Parameters Set: 857
Parameters Set: 858
Parameters Set: 859
Parameters Set: 860
Parameters Set: 861
Parameters Set: 862
Parameters Set: 863
Parameters Set: 864
Parameters Set: 865
Parameters Set: 866
Parameters Set: 867
Parameters Set: 868
Parameters Set: 869
Parameters Set: 870
Parameters Set: 871
Parameters Set: 872
Parameters Set: 873
Parameters Set: 874
Parameters Set: 875
Parameters Set: 876
Parameters Set: 877
Parameters Set: 878
Parameters Set: 879
Parameters Set: 880
Parameters Set: 881
Parameters Set: 882
Parameters Set: 883


Parameters Set: 1240
Parameters Set: 1241
Parameters Set: 1242
Parameters Set: 1243
Parameters Set: 1244
Parameters Set: 1245
Parameters Set: 1246
Parameters Set: 1247
Parameters Set: 1248
Parameters Set: 1249
Parameters Set: 1250
Parameters Set: 1251
Parameters Set: 1252
Parameters Set: 1253
Parameters Set: 1254
Parameters Set: 1255
Parameters Set: 1256
Parameters Set: 1257
Parameters Set: 1258
Parameters Set: 1259
Parameters Set: 1260
Parameters Set: 1261
Parameters Set: 1262
Parameters Set: 1263
Parameters Set: 1264
Parameters Set: 1265
Parameters Set: 1266
Parameters Set: 1267
Parameters Set: 1268
Parameters Set: 1269
Parameters Set: 1270
Parameters Set: 1271
Parameters Set: 1272
Parameters Set: 1273
Parameters Set: 1274
Parameters Set: 1275
Parameters Set: 1276
Parameters Set: 1277
Parameters Set: 1278
Parameters Set: 1279
Parameters Set: 1280
Parameters Set: 1281
Parameters Set: 1282
Parameters Set: 1283
Parameters Set: 1284
Parameters Set: 1285
Parameters Set: 1286
Parameters Se

Parameters Set: 1631
Parameters Set: 1632
Parameters Set: 1633
Parameters Set: 1634
Parameters Set: 1635
Parameters Set: 1636
Parameters Set: 1637
Parameters Set: 1638
Parameters Set: 1639
Parameters Set: 1640
Parameters Set: 1641
Parameters Set: 1642
Parameters Set: 1643
Parameters Set: 1644
Parameters Set: 1645
Parameters Set: 1646
Parameters Set: 1647
Parameters Set: 1648
Parameters Set: 1649
Parameters Set: 1650
Parameters Set: 1651
Parameters Set: 1652
Parameters Set: 1653
Parameters Set: 1654
Parameters Set: 1655
Parameters Set: 1656
Parameters Set: 1657
Parameters Set: 1658
Parameters Set: 1659
Parameters Set: 1660
Parameters Set: 1661
Parameters Set: 1662
Parameters Set: 1663
Parameters Set: 1664
Parameters Set: 1665
Parameters Set: 1666
Parameters Set: 1667
Parameters Set: 1668
Parameters Set: 1669
Parameters Set: 1670
Parameters Set: 1671
Parameters Set: 1672
Parameters Set: 1673
Parameters Set: 1674
Parameters Set: 1675
Parameters Set: 1676
Parameters Set: 1677
Parameters Se

...

Parameters Set: 18232
Parameters Set: 18233
Parameters Set: 18234
Parameters Set: 18235
Parameters Set: 18236
Parameters Set: 18237
Parameters Set: 18238
Parameters Set: 18239
Parameters Set: 18240
Parameters Set: 18241
Parameters Set: 18242
Parameters Set: 18243
Parameters Set: 18244
Parameters Set: 18245
Parameters Set: 18246
Parameters Set: 18247
Parameters Set: 18248
Parameters Set: 18249
Parameters Set: 18250
Parameters Set: 18251
Parameters Set: 18252
Parameters Set: 18253
Parameters Set: 18254
Parameters Set: 18255
Parameters Set: 18256
Parameters Set: 18257
Parameters Set: 18258
Parameters Set: 18259
Parameters Set: 18260
Parameters Set: 18261
Parameters Set: 18262
Parameters Set: 18263
Parameters Set: 18264
Parameters Set: 18265
Parameters Set: 18266
Parameters Set: 18267
Parameters Set: 18268
Parameters Set: 18269
Parameters Set: 18270
Parameters Set: 18271
Parameters Set: 18272
Parameters Set: 18273
Parameters Set: 18274
Parameters Set: 18275
Parameters Set: 18276
Parameters

Parameters Set: 18609
Parameters Set: 18610
Parameters Set: 18611
Parameters Set: 18612
Parameters Set: 18613
Parameters Set: 18614
Parameters Set: 18615
Parameters Set: 18616
Parameters Set: 18617
Parameters Set: 18618
Parameters Set: 18619
Parameters Set: 18620
Parameters Set: 18621
Parameters Set: 18622
Parameters Set: 18623
Parameters Set: 18624
Parameters Set: 18625
Parameters Set: 18626
Parameters Set: 18627
Parameters Set: 18628
Parameters Set: 18629
Parameters Set: 18630
Parameters Set: 18631
Parameters Set: 18632
Parameters Set: 18633
Parameters Set: 18634
Parameters Set: 18635
Parameters Set: 18636
Parameters Set: 18637
Parameters Set: 18638
Parameters Set: 18639
Parameters Set: 18640
Parameters Set: 18641
Parameters Set: 18642
Parameters Set: 18643
Parameters Set: 18644
Parameters Set: 18645
Parameters Set: 18646
Parameters Set: 18647
Parameters Set: 18648
Parameters Set: 18649
Parameters Set: 18650
Parameters Set: 18651
Parameters Set: 18652
Parameters Set: 18653
Parameters

Parameters Set: 18987
Parameters Set: 18988
Parameters Set: 18989
Parameters Set: 18990
Parameters Set: 18991
Parameters Set: 18992
Parameters Set: 18993
Parameters Set: 18994
Parameters Set: 18995
Parameters Set: 18996
Parameters Set: 18997
Parameters Set: 18998
Parameters Set: 18999
Parameters Set: 19000
Parameters Set: 19001
Parameters Set: 19002
Parameters Set: 19003
Parameters Set: 19004
Parameters Set: 19005
Parameters Set: 19006
Parameters Set: 19007
Parameters Set: 19008
Parameters Set: 19009
Parameters Set: 19010
Parameters Set: 19011
Parameters Set: 19012
Parameters Set: 19013
Parameters Set: 19014
Parameters Set: 19015
Parameters Set: 19016
Parameters Set: 19017
Parameters Set: 19018
Parameters Set: 19019
Parameters Set: 19020
Parameters Set: 19021
Parameters Set: 19022
Parameters Set: 19023
Parameters Set: 19024
Parameters Set: 19025
Parameters Set: 19026
Parameters Set: 19027
Parameters Set: 19028
Parameters Set: 19029
Parameters Set: 19030
Parameters Set: 19031
Parameters

Parameters Set: 19360
Parameters Set: 19361
Parameters Set: 19362
Parameters Set: 19363
Parameters Set: 19364
Parameters Set: 19365
Parameters Set: 19366
Parameters Set: 19367
Parameters Set: 19368
Parameters Set: 19369
Parameters Set: 19370
Parameters Set: 19371
Parameters Set: 19372
Parameters Set: 19373
Parameters Set: 19374
Parameters Set: 19375
Parameters Set: 19376
Parameters Set: 19377
Parameters Set: 19378
Parameters Set: 19379
Parameters Set: 19380
Parameters Set: 19381
Parameters Set: 19382
Parameters Set: 19383
Parameters Set: 19384
Parameters Set: 19385
Parameters Set: 19386
Parameters Set: 19387
Parameters Set: 19388
Parameters Set: 19389
Parameters Set: 19390
Parameters Set: 19391
Parameters Set: 19392
Parameters Set: 19393
Parameters Set: 19394
Parameters Set: 19395
Parameters Set: 19396
Parameters Set: 19397
Parameters Set: 19398
Parameters Set: 19399
Parameters Set: 19400
Parameters Set: 19401
Parameters Set: 19402
Parameters Set: 19403
Parameters Set: 19404
Parameters

Parameters Set: 19739
Parameters Set: 19740
Parameters Set: 19741
Parameters Set: 19742
Parameters Set: 19743
Parameters Set: 19744
Parameters Set: 19745
Parameters Set: 19746
Parameters Set: 19747
Parameters Set: 19748
Parameters Set: 19749
Parameters Set: 19750
Parameters Set: 19751
Parameters Set: 19752
Parameters Set: 19753
Parameters Set: 19754
Parameters Set: 19755
Parameters Set: 19756
Parameters Set: 19757
Parameters Set: 19758
Parameters Set: 19759
Parameters Set: 19760
Parameters Set: 19761
Parameters Set: 19762
Parameters Set: 19763
Parameters Set: 19764
Parameters Set: 19765
Parameters Set: 19766
Parameters Set: 19767
Parameters Set: 19768
Parameters Set: 19769
Parameters Set: 19770
Parameters Set: 19771
Parameters Set: 19772
Parameters Set: 19773
Parameters Set: 19774
Parameters Set: 19775
Parameters Set: 19776
Parameters Set: 19777
Parameters Set: 19778
Parameters Set: 19779
Parameters Set: 19780
Parameters Set: 19781
Parameters Set: 19782
Parameters Set: 19783
Parameters

In [4]:
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=NPSavgs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPSavg_ims\\fake_NPSavg_" + str(imnum + 1))
    plt.close(fig)

<ipython-input-4-47ca85929f0b>:5: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=NPSavgs[imnum].max())


In [5]:
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (49,49))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax = fakeNPSs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPS_ims\\fake_NPS_" + str(imnum + 1))
    plt.close(fig)

<ipython-input-5-bc15296044a8>:5: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax = fakeNPSs[imnum].max())


In [6]:
pkl.dump(NPSavgs, open(savepath + "fakeNPSavgs", "wb"))
pkl.dump(fakeNPSs, open(savepath + "fakeNPSs", "wb"))
pkl.dump(savedParas, open(savepath + "savedParas", "wb"))

In [7]:
fakeNPSs = np.array(fakeNPSs).reshape((len(fakeNPSs),-1))
savedParas = np.array(savedParas)
savedParas = np.delete(savedParas, np.where(np.amax(fakeNPSs, axis=1) > 20), 0)
fakeNPSs = np.delete(fakeNPSs, np.where(np.amax(fakeNPSs, axis=1) > 20), 0)
savedParas = np.delete(savedParas, np.where(np.amin(fakeNPSs, axis=1) < 0), 0)
fakeNPSs = np.delete(fakeNPSs, np.where(np.amin(fakeNPSs, axis=1) < 0), 0)
fakeNPSs /= 20
defocus = savedParas[:,[5]]
otherparas = np.delete(savedParas, 5, axis=1)
trainin, testin, trainout, testout, otherparasin, otherparasout = train_test_split(fakeNPSs, defocus, otherparas, test_size=0.25, shuffle=True)
# mean = np.mean(trainin)
# std = np.std(trainin)
# trainin -= mean
# trainin /= std
# testin -= mean
# testin /= std
# pkl.dump(mean, open(savepath + "mean", "wb"))
# pkl.dump(std, open(savepath + "std", "wb"))
for i in range(len(trainin)):
    trainin[i] -= np.mean(trainin[i])
    trainin[i] /= np.std(trainin[i])
for i in range(len(testin)):
    testin[i] -= np.mean(testin[i])
    testin[i] /= np.std(testin[i])
scaler = RobustScaler(unit_variance=True)
yscaler = scaler.fit(trainout)
pkl.dump(yscaler, open(savepath + "yscaler", "wb"))
trainout = yscaler.transform(trainout)
testout = yscaler.transform(testout)

In [8]:
# print(mean)
# print(std)

In [9]:
print(np.shape(fakeNPSs))

(100000, 2401)


In [10]:
trainin = tf.convert_to_tensor(trainin)
trainout = tf.convert_to_tensor(trainout)
testin = tf.convert_to_tensor(testin)
testout = tf.convert_to_tensor(testout)

In [11]:
pkl.dump(trainin, open(savepath + "trainin", "wb"))
pkl.dump(trainout, open(savepath + "trainout", "wb"))
pkl.dump(testin, open(savepath + "testin", "wb"))
pkl.dump(testout, open(savepath + "testout", "wb"))
pkl.dump(otherparasin, open(savepath + "otherparasin", "wb"))
pkl.dump(otherparasout, open(savepath + "otherparasout", "wb"))

In [12]:
NPSavgs = []
paras = []
for i in range(100):
    paras.append([0.29999999999999993, 1.0000000000000002, 1.3731274016165003e-31, 2.299291836656106, -1.9286236565392492, 0.5708288855841005, -0.11615444238292517])
for para in paras:
    NPSavgs.append(make_M2k_Fit(para, imgSysData))
fakeNPSs = NPSavg_To_Single(NPSavgs, numNPSs)

Parameters Set: 1
Parameters Set: 2
Parameters Set: 3
Parameters Set: 4
Parameters Set: 5
Parameters Set: 6
Parameters Set: 7
Parameters Set: 8
Parameters Set: 9
Parameters Set: 10
Parameters Set: 11
Parameters Set: 12
Parameters Set: 13
Parameters Set: 14
Parameters Set: 15
Parameters Set: 16
Parameters Set: 17
Parameters Set: 18
Parameters Set: 19
Parameters Set: 20
Parameters Set: 21
Parameters Set: 22
Parameters Set: 23
Parameters Set: 24
Parameters Set: 25
Parameters Set: 26
Parameters Set: 27
Parameters Set: 28
Parameters Set: 29
Parameters Set: 30
Parameters Set: 31
Parameters Set: 32
Parameters Set: 33
Parameters Set: 34
Parameters Set: 35
Parameters Set: 36
Parameters Set: 37
Parameters Set: 38
Parameters Set: 39
Parameters Set: 40
Parameters Set: 41
Parameters Set: 42
Parameters Set: 43
Parameters Set: 44
Parameters Set: 45
Parameters Set: 46
Parameters Set: 47
Parameters Set: 48
Parameters Set: 49
Parameters Set: 50
Parameters Set: 51
Parameters Set: 52
Parameters Set: 53
Pa

In [13]:
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=NPSavgs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPSavg_ims_SamePara\\fake_NPSavg_" + str(imnum + 1))
    plt.close(fig)

<ipython-input-13-52fb9dd46911>:5: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=NPSavgs[imnum].max())


In [14]:
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (49,49))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax = fakeNPSs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPS_ims_SamePara\\fake_NPS_" + str(imnum + 1))
    plt.close(fig)

<ipython-input-14-0351bb834365>:5: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax = fakeNPSs[imnum].max())


#### 